# Caso do Churrasco

Neste notebook o nosso objetivo é trabalhar modelagem e algoritmos de otimização através do problema prático do churrasco. Este problema parte da necessidade de comprar itens para realizar o churrasco de fim de ano, porém temos uma grande limitação: orçamento. Desta forma, queremos encontrar quais itens compramos de modo a utilizarmos a maior parte do dinheiro, obter a maior quantidade de alimentos e também atingir uma boa diversidade.

Como fica evidente, este problema é uma otimização multiobjetivo. Nossa função fitness (ou função objetivo) precisará lidar com esta particularidade e, em especial, permitir alterarmos a forma de compilar cada objetivo de forma isolada durante a execução dos algoritmos.

Trataremos neste notebook a concepção do problema, a modelagem, configuração, testes e por fim algoritmos de otimização. Resumidamente, os conceitos abordados serão:

- Organização de parâmetros de configuração de entrada do problema;
- Codificação de modelo para o problema utilizando OO;
- Formas de utilizar diversos objetivos para avaliar soluções;
- Implementar busca local com Algoritmo da Subida de Encosta;
- Solucionar problema com Algoritmo de Têmpera Simulada;
- Criar Algoritmo Genético para solucionar o problema;
- Adicionar etapa de explotação com busca local para aprimorar Algoritmo Genético

# Configuração de Ambiente

Nesta seção configuraremos todas as bibliotecas necessárias, bem como os caminhos para arquivos que serão utilizados. Este notebook faz parte de um repositório maior, portanto foi desenvolvido para trabalhar sinergicamente com os demais diretórios, em especial o de armazenamento de dados, onde está o arquivo de itens disponíveis para a compra. Caso deseje executar localmente, atente-se apenas para manter os diretórios do repositório original sem alteração, de modo que os comandos abaixo consigam encontrar os caminhos desejados.

Vamos trabalhar também com uma biblioteca auxiliar de tipagem, com o intuito de deixar mais claro o que são as variáveis trabalhadas em cada parte do código, sobretudo os argumentos de construtores e métodos. Desta forma, nosso código ficará mais legível e, mesmo depois de algum tempo sem nos debruçar sobre o código, conseguiremos entender o que cada variável significa. Outra grande funcionalidade é a possibilidade de utilizar o intellisense do editor de texto, o que é de grande ajuda quando estamos escrevendo código.

Por fim, utilizaremos o inglês para nomear variáveis. Esta é uma decisão puramente estética pessoal, para evitar a escrita "errada" de palavras em português quando precisarmos remover acentuações e caracteres especiais. Saiba que esta prática não representa nenhum tipo de convenção ou melhoria de performance, apenas se trata de uma forma de escrita pessoal.

In [1]:
import os
import json
import random
from enum import Enum
from typing import List, Tuple, Callable, Dict

BASE_PATH = os.path.dirname(os.getcwd())
ITEMS_PATH = f"{BASE_PATH}/data/churrasco.json"

# Criação do Modelo

O ponto de partida para começar a codificar é definir, textualmente, os requisitos que nosso modelo precisa entregar. Esta etapa serve como bússula para quando formos escrever o código, evitando que fiquemos travados diante da tela. Assim, são requisitos do modelo:

- Configurar e processar os itens que podem ser comprados;
- Registar informação de limite do orçamento disponível (restrição);
- Forma de representar a solução do problema, ou seja, itens a serem comprados;
- Procedimento de avaliar validade de uma solução, confrontando com o orçamento;
- Estratégia para encontrar soluções vizinhas, dada uma solução qualquer;
- Método de avaliação da solução a partir de regras específicas (função fitness).

## Itens disponíveis

Para configurar os itens de entradas utilizaremos um arquivo JSON como exemplificado abaixo. A estratégia deste JSON é configurar itens de forma genérica, introduzindo o conceito de compra por lote. Carnes geralmente são compradas por kg, então sua "Unidades/Lote" possui valor *null*. Por outro lado, o pão de alho é comprado por lotes de 10 unidades. O campo "Incremento" informa de quanto em quanto um item pode ser comprado. Novamente, carnes poderiam ser compradas em qualquer quantidade, porém não é prático pedir frações menores que 100g. Por outro lado, o pão de alho sempre é comprado de lote em lote, ou seja de 1 em 1. Os campos "Preço/Lote" e "kg/Lote" são autoexplicativos, indicam o preço pago por um lote do item (para as carnes um lote é 1kg e para o pão de alho o lote são 10 pães) e quanto de massa o lote tem, respectivamente.
```
{
    "Alcatra": {
        "Preço/Lote": 47.99,
        "Unidades/Lote": null,
        "kg/Lote": 1,
        "Incremento": 0.1
    },
    "Pão de Alho": {
        "Preço/Lote": 15.97,
        "Unidades/Lote": 10,
        "kg/Lote": 0.8,
        "Incremento": 1
    },
    "Picanha": {...},
    "Pernil": {...},
    ...
}
```

Com o JSON estruturado, vamos criar a classe responsável por transcrever esta estrutura de dados. Esta será a classe utilizada pelo modelo para gerenciar os itens disponíveis para compra, evitando trabalhar com strings de chave e valor, acessando o JSON (transformado em dicionário no python.).

In [2]:
class Item:
    def __init__(self, name: str, batch_price: float,
        increment: float | None, batch_kg: float
    ) -> None:
        self.name = name
        self.batch_kg = batch_kg
        self.increment = increment
        self.batch_price = batch_price

## Processamento do arquivo

O método de carga **process_json** é responsável por ler o arquivo JSON convertido para dicionário, criando as instâncias de Item para cada item cadastrado no arquivo. Assim, alterações de preço ou de possibilidades de compra ficam isoladas do código, sendo manipuladas apenas no arquivo. Como estas informações não são interessantes de serem alteradas para teste, não é ruim deixar estático no JSON.

Lembre-se, você pode alterar os itens disponíveis e preços para atender à sua realidade, caso deseje. Basta alterar o arquivo no diretório específico, mantendo apenas a estrutura de atributos apresentada anteriormente.

## Representação da solução, restrição e validação

A forma mais simples de representar uma solução é por meio de listas, em que cada índice corresponde a quantidade do item, de modo que o índice é o mesmo no atributo interno gerado após o processamento do arquivo. Como a solução é manipulada apenas dentro da classe, não se espera problemas de desconexão entre índices de cada item, além de permitir melhor dinamicidade na geração da solução, que fica condicionada ao tamanho de itens cadastrados no arquivo JSON.

Este problema possui uma única restrição, que é o orçamento máximo disponível. Como é interessante nos próximos passos realizar testes com diferentes orçamentos, esta restrição foi passada dentro do construtor, lida durante a execução do código. Também seria possível mover as restrições para um arquivo, porém neste caso optamos por trabalhar com a passagem de restrição durante o código, dada as particularidades do caso.

Por fim, existindo uma solução e a restrição, o método **valid_solution** é responsável por verificar se a solução é viável. Em outras palavras, verifica se ela atende às restrições existentes. No nosso caso, a única restrição é se as quantidades compradas não extrapolam o valor disponível no orçamento.

## Soluções aleatórias e vizinhas

Todos os algoritmos de otimização precisam de uma estimativa inicial, portanto o método **random_solution** é responsável por entregar esta funcionalidade. Outra funcionalidade são as soluções vizinhas, suprida pelo método **near_solution**. Para este método, a estratégia foi utilizar o atributo de incremento, uma vez que todas os itens foram quantizados. Porém, para deixar genérico, um *if* regula o caso de incremento *null*, fazendo com que ele se torne 5% do valor atual.

## Função objetivo

Como o nosso problema é multiobjetivo, vamos criar um enum para listar os diferentes objetivos que serão trabalhados. Cada objetivo vai ser um item no enum, que será utilizado para compor a função objetivo de modo dinâmico. A estratégia mais simples para se trabalhar com funções multiobjetivo é atribuir pesos para cada objetivo e então computar o valor final como a média ponderada. Porém existe uma atenção para se aplicar esta abordagem: as dimensões e unidades precisam ser equivalentes.

Neste caso vamos trabalhar com três funções objetivo, todas que variam entre 0 e 1. São elas:
- Porcentagem de gasto do orçamento;
- Porcentagem de massa comprada;
- Índice de variedade.

A porcentagem de gasto do orçamento é simples, basta dividir o total gasto com a solução pelo orçamento disponível. A porcentagem de massa comprada exige um trabalho extra, precisamos encontrar a massa mássima que poderíamos comprar com o orçamento disponível. Para isso, basta encontrar o produto com menor R$/kg e verificar o quanto seria possível comprar de massa dele. Este cálculo é feito quando o processamento do arquivo JSON é realizado e a massa mássima (referência para o cálculo deste objetivo) é armazenado no atributo da classe. Por fim, o índice de variedade é uma definição nossa. Vamos definir como a razão entre a menor fração mássica da solução e a fração mássica no caso de todos os itens serem comprados de forma igualitária (em massa).

Importante pontuar que o índice de variedade é uma medida que chamados de proxy. Não existe um índice de variedade, então este indicador que comentamos é uma forma de aproximar a ideia de diversidade de itens comprados. Precisamos que ele atenda a ideia de ser zero quando existe algum item não comprado (0 na solução) e que aumente de acordo com o aumento da quantidade comprada. Esta forma de cálculo não é única, poderíamos definir outro índice de variedade como sendo $ \sum_{i=1}^n P_i \over n $, em que n é o número de itens disponíveis e $ P_i $ é uma variável binária que assume o valor 1 se a quantidade do item for maior que zero e assume 0 caso contrário.

In [20]:
class Objective(Enum):
    MASSA_TOTAL = 1
    GASTO_TOTAL = 2
    DIVERSIDADE = 3


class Barbecue:
    def __init__(self, money_limit: float) -> None:
        self.items: List[Item] = []
        self.mass_limit: float = None
        self.money_limit = money_limit
        self.relate_objective: Dict[Objective, Callable[[List[float]], float]] = {
            Objective.MASSA_TOTAL: self.calculate_mass,
            Objective.GASTO_TOTAL: self.calculate_spent,
            Objective.DIVERSIDADE: self.calculate_variety
        }

    def random_solution(self) -> List[float]:
        while True:
            solution: List[float] = []
            for i in range(0, len(self.items)):
                multiply = random.randint(0, 10)
                quantity = round(self.items[i].increment * multiply, 1)
                solution.append(quantity)

            if self.valid_solution(solution):
                break

        return solution

    def near_solution(self, solution: List[float], quantity: int) -> List[List[float]]:
        new_solutions: List[List[float]] = []
        
        for _ in range(0, quantity):
            while True:
                new_solution = solution.copy()
                i = random.randint(0, len(solution) - 1)
                signal = 1 if random.random() < 0.5 else -1
                step = self.items[i].increment if self.items[i].increment is not None else 0.05 * new_solution[i]
                new_solution[i] = round(new_solution[i] + signal * step, 1)

                if self.valid_solution(new_solution):
                    break

            new_solutions.append(new_solution)
        
        return new_solutions

    def calculate_fitness(self, solution: List[float],
        compose: List[Tuple[Objective, float]]
    ) -> float:
        score, weights = 0, 0
        for objective, weight in compose:
            fitness = self.relate_objective[objective](solution)
            score = score + fitness * weight
            weights = weights + weight
        return score / weights

    def calculate_spent(self, solution: List[float]) -> float:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]
        return spent / self.money_limit
    
    def calculate_mass(self, solution: List[float]) -> float:
        mass = 0
        for i in range(0, len(solution)):
            mass = mass + self.items[i].batch_kg * solution[i]
        return mass / self.mass_limit
    
    def calculate_variety(self, solution: List[float]) -> float:
        # Encontra a massa de cada item
        mass = []
        for i in range(0, len(solution)):
            mass.append(self.items[i].batch_kg * solution[i])
        
        # Calcula a fração mássica de cada item
        mass_total = sum(mass)
        frac = [m / mass_total for m in mass]

        # Define diversidade como a razão entre a menor fração
        # mássica e a fração mássica em condições totalmente
        # iguais para todos os itens
        equals_frac = 1 / len(solution)
        return min(frac) / equals_frac

    def process_json(self, data: dict) -> None:
        # Carrega itens do json
        items: List[Item] = []
        for item in data.keys():
            items.append(Item(
                item,
                data[item]["Preço/Lote"],
                data[item]["Incremento"],
                data[item]["Massa/Lote"]
            ))
        self.items = items

        # Calcula a massa máxima que pode ser obtida, resultado
        # do gasto de todo o orçamento no item de menor R$/kg
        max_mass = 0
        for item in self.items:
            spent, multiply = 0, 0
            while spent < self.money_limit:
                spent = spent + item.increment * item.batch_price
                multiply = multiply + item.increment

            # Remova um incremento, caso não queira ultrapassar
            # o orçamento disponível
            # multiply = multiply - item.increment
            item_mass = multiply * item.batch_kg
            if item_mass > max_mass:
                max_mass = item_mass

        self.mass_limit = max_mass

    def valid_solution(self, solution: List[float]) -> bool:
        spent = 0
        for i in range(0, len(solution)):
            spent = spent + self.items[i].batch_price * solution[i]

        if spent > self.money_limit:
            return False
        else:
            return True
        
    def print_solution(self, solution: List[float]) -> None:
        print("----Apresentando especificidades da solução----")
        print(f"Total gasto: {round(self.calculate_spent(solution), 3) * 100}%")
        print(f"Massa atingida: {round(self.calculate_mass(solution), 3) * 100}%")
        print(f"Índice de variedade: {round(self.calculate_variety(solution), 2)}")
        print()

        for i in range(0, len(solution)):
            spent = self.items[i].batch_price * solution[i]
            mass = self.items[i].batch_kg * solution[i]
            print(f"{self.items[i].name}: R$ {round(spent, 2)} ({round(mass, 1)} kg)")

## Executando o Modelo

Os blocos abaixo possuem função ilustrativa, para visualizar a execução dos métodos criados. É importante, além de desenvolver, ter uma seção para testar o que foi feito, como é a interação dos métodos e se tudo está funcionando como deveria. Em aplicações mais robustas, um módulo de testes é muito bem-vindo, mas aqui vamos simplificar um pouco, já que o objetivo não é colocar o modelo em produção, mas sim executar um trabalho de otimização.

In [21]:
# Carregando o arquivo JSON
with open(ITEMS_PATH) as file:
    data = json.load(file)

barbecue = Barbecue(money_limit=300)
barbecue.process_json(data)

In [29]:
# Encontrando uma solução aleatória
solution = barbecue.random_solution()
barbecue.print_solution(solution)

----Apresentando especificidades da solução----
Total gasto: 82.0%
Massa atingida: 47.3%
Índice de variedade: 0.13

Alcatra: R$ 47.99 (1.0 kg)
Linguiça: R$ 11.99 (0.6 kg)
Pernil: R$ 9.59 (0.6 kg)
Lombo: R$ 3.4 (0.2 kg)
Picanha: R$ 61.19 (0.9 kg)
Pão de Alho: R$ 111.79 (5.6 kg)


In [32]:
# Calculando o fitness final, compondo os pesos de cada objetivo
compose = [
    (Objective.MASSA_TOTAL, 0.3),
    (Objective.GASTO_TOTAL, 0.5),
    (Objective.DIVERSIDADE, 0.2)
]
fitness_total = barbecue.calculate_fitness(solution, compose)
print(f"Fitness geral: {round(fitness_total, 3)}")

Fitness geral: 0.579


In [34]:
# Encontra vizinhos da solução
neighbors = barbecue.near_solution(solution, quantity=10)
print(f"Representação da solução inicial:\n{solution}")
print()
for n in neighbors:
    print(f"Vizinho: {n}")

Representação da solução inicial:
[1.0, 0.6, 0.6, 0.2, 0.9, 7]

Vizinho: [1.0, 0.6, 0.6, 0.2, 0.9, 6]
Vizinho: [1.0, 0.6, 0.5, 0.2, 0.9, 7]
Vizinho: [1.0, 0.6, 0.6, 0.1, 0.9, 7]
Vizinho: [1.0, 0.6, 0.6, 0.3, 0.9, 7]
Vizinho: [1.0, 0.5, 0.6, 0.2, 0.9, 7]
Vizinho: [1.0, 0.6, 0.6, 0.1, 0.9, 7]
Vizinho: [1.0, 0.6, 0.5, 0.2, 0.9, 7]
Vizinho: [0.9, 0.6, 0.6, 0.2, 0.9, 7]
Vizinho: [1.0, 0.6, 0.6, 0.2, 0.8, 7]
Vizinho: [0.9, 0.6, 0.6, 0.2, 0.9, 7]
